In [14]:
import os

os.environ["KAGGLE_USERNAME"] = "joshideepak08"
os.environ["KAGGLE_API_KEY"] = "KGAT_ad5043e2d749348bd7ecb872480b8f19"

print("Username:", os.environ.get("KAGGLE_USERNAME"))
print("Key exists:", os.environ.get("KAGGLE_API_KEY") is not None)


Username: joshideepak08
Key exists: True


In [15]:
!mkdir -p ~/.kaggle
!echo '{"username":"joshideepak08","key":"KGAT_ad5043e2d749348bd7ecb872480b8f19"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [18]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sakshamjn/vehicle-detection-8-classes-object-detection")

print("Path to dataset files:", path)

100%|██████████| 346M/346M [00:03<00:00, 111MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sakshamjn/vehicle-detection-8-classes-object-detection/versions/1


In [19]:
!mkdir -p data/vehicle_dataset
!cp -r /root/.cache/kagglehub/datasets/sakshamjn/vehicle-detection-8-classes-object-detection/versions/1/* data/vehicle_dataset


In [21]:
!ls data/vehicle_dataset/train


images	labels	labels.npy


In [23]:
!find data/vehicle_dataset/train -type f | head -n 20


data/vehicle_dataset/train/images/Highway_545_2020-07-30_jpg.rf.aac4ce2b6f8a145375f202b79c66a792.jpg
data/vehicle_dataset/train/images/highway_3359_2020-08-26.jpg
data/vehicle_dataset/train/images/ulu2450_jpg.rf.261c58216d131a33b9a3e379a940bcde.jpg
data/vehicle_dataset/train/images/Highway_1236_2020-07-30_jpg.rf.ffca5dabfa87a2928053be64d92c6e59.jpg
data/vehicle_dataset/train/images/ulu1534_jpg.rf.ed32678a013a30f516624b8be4b9bca4.jpg
data/vehicle_dataset/train/images/highway_3583_2020-08-26.jpg
data/vehicle_dataset/train/images/Highway_853_2020-07-30_jpg.rf.b56c050ec0cb16f7443a1fe21500efd5.jpg
data/vehicle_dataset/train/images/Highway_1981_2020-07-30_jpg.rf.70cfad3a7189480cd5d3eb4412a6c979.jpg
data/vehicle_dataset/train/images/Highway_1285_2020-07-30_jpg.rf.b79a7ab876471f7952f7654f66de6425.jpg
data/vehicle_dataset/train/images/highway_3712_2020-08-26.jpg
data/vehicle_dataset/train/images/Highway_869_2020-07-30_jpg.rf.7e13cb419157a87c2a2b9439328aeea8.jpg
data/vehicle_dataset/train/images

In [52]:
import torch
import os

# -------- CONFIG --------
DATA_ROOT = "data/vehicle_dataset/train"
MAX_IMAGES = 2000        # speed + memory safe
NUM_EPOCHS = 3
BATCH_SIZE = 2           # OOM-safe
LR = 1e-3
MAX_IMAGE_SIZE = 800     # resize large images

torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [53]:
import cv2
from torch.utils.data import Dataset

CLASSES = {
    0: "car",
    1: "bus",
    2: "truck",
    3: "motorcycle",
    4: "bicycle"
}

class VehicleDataset(Dataset):
    def __init__(self, root_dir, classes, max_images=None):
        self.image_dir = os.path.join(root_dir, "images")
        self.label_dir = os.path.join(root_dir, "labels")
        self.classes = classes

        self.images = sorted(os.listdir(self.image_dir))
        if max_images:
            self.images = self.images[:max_images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        # resize large images (VERY IMPORTANT)
        if max(h, w) > MAX_IMAGE_SIZE:
            scale = MAX_IMAGE_SIZE / max(h, w)
            image = cv2.resize(image, (int(w*scale), int(h*scale)))
            h, w, _ = image.shape

        label_path = os.path.join(
            self.label_dir, img_name.replace(".jpg", ".txt")
        )

        boxes, labels = [], []

        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f:
                    cls, xc, yc, bw, bh = map(float, line.split())
                    if int(cls) not in self.classes:
                        continue

                    xmin = (xc - bw/2) * w
                    ymin = (yc - bh/2) * h
                    xmax = (xc + bw/2) * w
                    ymax = (yc + bh/2) * h

                    boxes.append([xmin, ymin, xmax, ymax])
                    labels.append(int(cls) + 1)  # +1 background

        if len(boxes) == 0:
            return None

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64)
        }

        image = torch.tensor(image/255., dtype=torch.float32).permute(2, 0, 1)
        return image, target


In [54]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None
    return tuple(zip(*batch))

dataset = VehicleDataset(
    DATA_ROOT,
    CLASSES,
    max_images=MAX_IMAGES
)

train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

print("Total training images:", len(dataset))


Total training images: 2000


In [55]:
import torch.nn as nn

class CustomBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.body = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.out_channels = 256

    def forward(self, x):
        return self.body(x)


In [56]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = CustomBackbone()

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

model = FasterRCNN(
    backbone=backbone,
    num_classes=len(CLASSES) + 1,
    rpn_anchor_generator=anchor_generator,
    rpn_pre_nms_top_n_train=600,
    rpn_post_nms_top_n_train=300
)

model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): CustomBackbone(
    (body): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU()
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(256, ep

In [59]:
from torch.cuda.amp import autocast, GradScaler

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scaler = GradScaler()


/tmp/ipython-input-542241706.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [60]:
from tqdm import tqdm

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0

    for batch in tqdm(train_loader):
        if batch is None:
            continue

        images, targets = batch

        valid_images, valid_targets = [], []
        for img, tgt in zip(images, targets):
            if tgt["boxes"].numel() == 0:
                continue
            valid_images.append(img)
            valid_targets.append(tgt)

        if len(valid_targets) == 0:
            continue

        images = [img.to(device) for img in valid_images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in valid_targets]

        optimizer.zero_grad()

        with autocast():
            loss_dict = model(images, targets)
            loss = sum(loss_dict.values())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

        del loss_dict, loss
        torch.cuda.empty_cache()

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | Loss: {epoch_loss:.4f}")


  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipython-input-2744900887.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 1000/1000 [04:37<00:00,  3.60it/s]


Epoch [1/3] | Loss: 799.1042


100%|██████████| 1000/1000 [04:13<00:00,  3.94it/s]


Epoch [2/3] | Loss: 669.5180


100%|██████████| 1000/1000 [04:13<00:00,  3.94it/s]

Epoch [3/3] | Loss: 687.7200


In [61]:
torch.save(model.state_dict(), "faster_rcnn_vehicle_from_scratch.pth")
!ls -lh faster_rcnn_vehicle_from_scratch.pth


-rw-r--r-- 1 root root 57M Jan 11 20:45 faster_rcnn_vehicle_from_scratch.pth


In [62]:
model.eval()
img, _ = dataset[5]
img = img.to(device)

with torch.no_grad():
    out = model([img])

print(out[0].keys())


dict_keys(['boxes', 'labels', 'scores'])


In [63]:
def compute_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])

    union = area1 + area2 - inter
    return inter / union if union > 0 else 0


In [66]:
def evaluate_map(
    model,
    dataset,
    iou_thresh=0.5,
    score_thresh=0.5,
    samples=200
):
    model.eval()
    correct = 0
    total = 0
    checked = 0
    idx = 0

    while checked < samples and idx < len(dataset):
        sample = dataset[idx]
        idx += 1

        # 🔒 Skip invalid samples
        if sample is None:
            continue

        img, target = sample
        img = img.to(device)

        with torch.no_grad():
            pred = model([img])[0]

        gt_boxes = target["boxes"]
        pred_boxes = pred["boxes"].cpu()
        scores = pred["scores"].cpu()

        for pb, s in zip(pred_boxes, scores):
            if s < score_thresh:
                continue

            total += 1
            for gb in gt_boxes:
                if compute_iou(pb, gb) >= iou_thresh:
                    correct += 1
                    break

        checked += 1

    return correct / total if total > 0 else 0


In [67]:
map_50 = evaluate_map(model, dataset)
print("Approx mAP@0.5:", map_50)


Approx mAP@0.5: 0.7692307692307693


In [68]:
import time
import torch

model.eval()

# get one valid sample
idx = 0
while True:
    sample = dataset[idx]
    idx += 1
    if sample is not None:
        image, _ = sample
        break

image = image.to(device)

# ---- GPU WARM-UP (VERY IMPORTANT) ----
for _ in range(5):
    with torch.no_grad():
        _ = model([image])

# ---- ACTUAL FPS MEASUREMENT ----
start = time.time()
with torch.no_grad():
    _ = model([image])
end = time.time()

fps = 1 / (end - start)

print(f"Inference Speed (FPS): {fps:.2f}")


Inference Speed (FPS): 9.92


In [69]:
!ls -lh faster_rcnn_vehicle_from_scratch.pth


-rw-r--r-- 1 root root 57M Jan 11 20:45 faster_rcnn_vehicle_from_scratch.pth


In [75]:
import cv2
import os
import torch
import numpy as np
from tqdm import tqdm

model.eval()

# ---------- VIDEO SETTINGS ----------
OUTPUT_VIDEO = "faster_rcnn_demo.mp4"
FPS = 10
SCORE_THRESHOLD = 0.5
MAX_FRAMES = 500   # short demo (10–20 sec)

# ---------- GET IMAGE LIST ----------
image_files = dataset.images[:MAX_FRAMES]

# ---------- READ FIRST IMAGE FOR SIZE ----------
first_img_path = os.path.join(DATA_ROOT, "images", image_files[0])
frame = cv2.imread(first_img_path)
h, w, _ = frame.shape

# ---------- VIDEO WRITER ----------
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(
    OUTPUT_VIDEO, fourcc, FPS, (w, h)
)

print("Creating demo video...")

# ---------- RUN INFERENCE FRAME BY FRAME ----------
for img_name in tqdm(image_files):
    img_path = os.path.join(DATA_ROOT, "images", img_name)

    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    img_tensor = torch.tensor(
        image_rgb / 255.0,
        dtype=torch.float32
    ).permute(2, 0, 1).to(device)

    with torch.no_grad():
        output = model([img_tensor])[0]

    boxes = output["boxes"].cpu().numpy()
    scores = output["scores"].cpu().numpy()
    labels = output["labels"].cpu().numpy()

    # ---------- DRAW BOXES ----------
    for box, score, label in zip(boxes, scores, labels):
        if score < SCORE_THRESHOLD:
            continue

        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        text = f"ID {label} | {score:.2f}"
        cv2.putText(
            image, text,
            (x1, max(y1 - 10, 10)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (0, 255, 0), 2
        )

    video_writer.write(image)

video_writer.release()
print("Video saved at:", OUTPUT_VIDEO)


Creating demo video...


100%|██████████| 500/500 [00:41<00:00, 12.01it/s]

Video saved at: faster_rcnn_demo.mp4
